In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alexaamazonds/EcoPreprocessed.csv


In [3]:
import pandas as pd
df = pd.read_csv("/kaggle/input/alexaamazonds/EcoPreprocessed.csv")
print(df.head(10))

   Unnamed: 0                                             review  polarity  \
0        3870                            able play youtube alexa    0.5000   
1          62  able recognize indian accent really well drop ...    0.2794   
2         487  absolute smart device amazon connect external ...    0.1827   
3        3204  absolutely amaze new member family control hom...    0.3682   
4        1265  absolutely amaze previously sceptical invest m...    0.2333   
5          53  absolutely cheat customer if buy amazon produc...    0.1350   
6         257  absolutely house hold item fair price intuitiv...    0.4250   
7        1258  absolutely mind blow device beautiful shape ex...    0.5262   
8        3499                              accept dutch language    0.0000   
9        2578  accessibility limit amazon apps music appsexce...   -0.0905   

   division  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  
5  positive  
6  positive  
7  positive  
8   neutral  
9

In [5]:

print(df.shape)

print(df.columns)

print(df.isnull().sum())

# Display random sample
df.sample(5)

(4084, 4)
Index(['Unnamed: 0', 'review', 'polarity', 'division'], dtype='object')
Unnamed: 0    0
review        0
polarity      0
division      0
dtype: int64


,Unnamed: 0,review,polarity,division
156,3075,alexa really dosent hear clearly understand ex...,0.2875,positive
954,3969,excellant,0.0000,neutral
3182,212,question answer whats weather today setupfor e...,-0.1204,negative
752,3042,device awesomeproducts quality feature great,0.6500,positive
1620,3140,good product buy,0.7000,positive


Se have 2 Datasets. One on 4000 row. Another one on 30 000+ rows. So we will test on a first one?

In [6]:
splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
df_hf_multi_train = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["train"])

print(df_hf_multi_train.shape)

print(df_hf_multi_train.columns)

print(df_hf_multi_train.isnull().sum())

# Display random sample
df_hf_multi_train.sample(5)

(31232, 4)
Index(['id', 'text', 'label', 'sentiment'], dtype='object')
id           0
text         0
label        0
sentiment    0
dtype: int64


,id,text,label,sentiment
29686,23019,http://twitpic.com/4jfkw - AFC! Oh yess,2,positive
22639,22331,i miss 'mr.',0,negative
14974,4296,While i can understand limiting somethings for...,0,negative
7296,12624,Going to get my blood taken,1,neutral
914,22938,haha..yeah..10th of May..lol..im greeting in ...,2,positive


In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

2025-08-08 08:02:59.293181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754640179.534396      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754640179.609704      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
df['division'].value_counts()

division
positive    3066
neutral      536
negative     482
Name: count, dtype: int64

In [9]:
df_hf_multi_train['sentiment'].value_counts()

sentiment
neutral     11649
positive    10478
negative     9105
Name: count, dtype: int64

The Second Dataset would be much better. It`s more evenly distributed.

In [10]:
df_hf_multi_val = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["validation"])
df_hf_multi_test = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["test"])
print(df_hf_multi_val.shape)
print(df_hf_multi_test.shape)

(5205, 4)
(5206, 4)


We will drop redundant columns: id and sentiment. (Sentiment because labe is already sentiment ==> 2 = positive; 0 = negative; 1 = neutral)

In [15]:
#display(df_hf_multi_train.drop('id', axis = 1))
#label_map = {'positive': 2, 'neutral': 1, ,'negative': 0}
#df_hf_multi_train['label'] = df_hf_multi_train['sentiment'].map(label_map)

df_hf_multi_train = df_hf_multi_train.drop(columns=['id'])
df_hf_multi_train = df_hf_multi_train.drop(columns=['sentiment'])
display(df_hf_multi_train.head())


,text,label
0,"Cooking microwave pizzas, yummy",2
1,Any plans of allowing sub tasks to show up in ...,1
2,"I love the humor, I just reworded it. Like sa...",2
3,naw idk what ur talkin about,1
4,That sucks to hear. I hate days like that,0


Distribution for Test and Validation sets

In [16]:
print(df_hf_multi_test.shape)

print(df_hf_multi_test.columns)

print(df_hf_multi_test.isnull().sum())

# Display random sample
df_hf_multi_test.sample(5)

(5206, 4)
Index(['id', 'text', 'label', 'sentiment'], dtype='object')
id           0
text         1
label        0
sentiment    0
dtype: int64


,id,text,label,sentiment
3489,4220,"Do not pay for the app. Sync doesn't work , th...",0,negative
565,13831,"happy mothers day to all the mothers, grandmot...",2,positive
1322,10102,Very good tool to organize my days .. thank,2,positive
2535,14221,Welcome back to Japan,1,neutral
1605,18750,bye! off to the movies with jake,1,neutral


In [17]:
print(df_hf_multi_val.shape)

print(df_hf_multi_val.columns)

print(df_hf_multi_val.isnull().sum())

# Display random sample
df_hf_multi_val.sample(5)

(5205, 4)
Index(['id', 'text', 'label', 'sentiment'], dtype='object')
id           0
text         0
label        0
sentiment    0
dtype: int64


,id,text,label,sentiment
2987,9448,Need some coffee... Getting serious withdrawal...,0,negative
3173,2135,"Well, there is THAT...",1,neutral
2234,194,I`m starving!! This diet is killing me but I c...,0,negative
5107,2324,to overcome whatever darts the enemy may try a...,2,positive
2280,9658,"feeling very stressed and strung out, awake at...",0,negative


In [18]:
# Show all rows with at least one NaN value
df_hf_multi_test[df_hf_multi_test.isnull().any(axis=1)]

,id,text,label,sentiment
3751,314,NaN,1,neutral


In [19]:
#check specific row
df_hf_multi_test.loc[3751]

id               314
text             NaN
label              1
sentiment    neutral
Name: 3751, dtype: object

In [20]:
df_hf_multi_test[df_hf_multi_test['text'].isnull()]

,id,text,label,sentiment
3751,314,NaN,1,neutral


In [21]:
#df_hf_multi_test = df_hf_multi_test[df_hf_multi_test['text'].notnull()]

df_hf_multi_test = df_hf_multi_test.dropna()

#df_hf_multi_test = df_hf_multi_test.drop(index=3751, errors='ignore')
#If your DataFrame has a default integer index and you want to reset it after deletion:
#df_hf_multi_test = df_hf_multi_test.reset_index(drop=True)

In [22]:
print(df_hf_multi_test.isnull().sum())

id           0
text         0
label        0
sentiment    0
dtype: int64


We want to delete redundant columns for Test and Validation dataset?